In [127]:
using Distributions
using Serialization
using CSV
using Distributed
using ProgressMeter
include("gen_crp_softmax.jl")
# Set random seed for reproducibility
Random.seed!(1777)
n_subs=100


100

In [142]:
alpha = 0.0
gamma_base = 1
gamma_coef = 0
beta = 5
# Create array to store results from each worker
p = Progress(n_subs, desc="Simulating subjects...")
results = @distributed (vcat) for sub_num in 1:n_subs
    simulated_data, max_clus, total_reward = crp([alpha, gamma_base, gamma_coef, beta])
    simulated_data[!, :sub_num] .= sub_num
    next!(p)
    simulated_data
end

# Save simulated data to CSV
CSV.write("results/model_predictions/simulated_data_non_structure_learner.csv", all_simulated_data)




Simulating subjects... 100%|█████████████████████████████| Time: 0:09:37


"results/model_predictions/simulated_data_non_structure_learner.csv"

In [143]:
alpha = 0.2
gamma_base = 1
gamma_coef = 0
beta = 5

# Create array to store results from each worker
p = Progress(n_subs, desc="Simulating subjects...")
results = @distributed (vcat) for sub_num in 1:n_subs
    simulated_data, max_clus, total_reward = crp([alpha, gamma_base, gamma_coef, beta])
    simulated_data[!, :sub_num] .= sub_num
    next!(p)
    simulated_data
end
# Save simulated data to CSV
CSV.write("results/model_predictions/simulated_data_structure_learner.csv", results)

Simulating subjects... 100%|█████████████████████████████| Time: 0:09:57


"results/model_predictions/simulated_data_structure_learner.csv"

In [144]:
# Add switch column based on galaxy transitions
function add_switch_column(data)
    switch_col = []
    for i in 1:nrow(data)
        planet = data.planet_num[i]
        curr_planet_type = data.galaxy[i]
        
        if planet > 0
            last_planet_type = data[data.planet_num .== (planet-1), :galaxy][1]
            if curr_planet_type == last_planet_type
                push!(switch_col, String("no_switch"))
            else
                push!(switch_col, String("switch"))
            end
        else
            push!(switch_col, String("no_switch"))
        end
    end
    return switch_col
end

add_switch_column (generic function with 1 method)

In [145]:
alpha = 0.2
gamma_base = 1
gamma_coef = 0
beta = 5
# Create array to store results from each worker
p = Progress(n_subs, desc="Simulating subjects...")
results = @distributed (vcat) for sub_num in 1:n_subs
    simulated_data, max_clus, total_reward = crp([alpha, gamma_base, gamma_coef, beta])
    switch_col = add_switch_column(simulated_data)
    insertcols!(simulated_data, :switch => switch_col)
    simulated_data[!, :sub_num] .= sub_num
    next!(p)
    simulated_data
end
# Save simulated data to CSV
CSV.write("results/model_predictions/simulated_data_uncertainty_non_adapter.csv", results)


Simulating subjects... 100%|█████████████████████████████| Time: 0:10:01


"results/model_predictions/simulated_data_uncertainty_non_adapter.csv"

In [148]:
alpha = 0.2
gamma_base = 1
gamma_coef = -0.3
beta = 5
# Create array to store results from each worker
p = Progress(n_subs, desc="Simulating subjects...")
results = @distributed (vcat) for sub_num in 1:n_subs
    simulated_data, max_clus, total_reward = crp([alpha, gamma_base, gamma_coef, beta])
    switch_col = add_switch_column(simulated_data)
    insertcols!(simulated_data, :switch => switch_col)
    simulated_data[!, :sub_num] .= sub_num
    next!(p)
    simulated_data
end
# Save simulated data to CSV
CSV.write("results/model_predictions/simulated_data_uncertainty_adapter.csv", results)


Simulating subjects... 100%|█████████████████████████████| Time: 0:09:29


"results/model_predictions/simulated_data_uncertainty_adapter.csv"